In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.20/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\13479\.wdm\drivers\chromedriver\win32\87.0.4280.20]


In [5]:
# wikipedia best seller URl of the page to be scrapped
best_seller_url = 'https://en.m.wikipedia.org/wiki/Publishers_Weekly_list_of_bestselling_novels_in_the_United_States_in_the_1910s'

browser.visit(best_seller_url)

html = browser.html

books_soup = BeautifulSoup(html, 'html.parser')

In [5]:
# collect the latest News Title and Paragraph Text
news_title = books_soup.find_all('div', class_='content_title')[0].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text
mars["news_title"] = news_title
mars["news_p"] = news_p
print(news_title)
print("--------------------------------------------------------------------")
print(news_p)


Mars Now
--------------------------------------------------------------------
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [6]:
# JPL Mars Space Images - Featured Image
# Mars Image to be scraped
jpl_nasa_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(jpl_nasa_url)
time.sleep(5)
browser.find_by_id("full_image").click()
time.sleep(5)
browser.find_link_by_partial_text("more info").click()
html = browser.html

images_soup = BeautifulSoup(html, 'html.parser')
image_link = images_soup.find('figure',class_="lede")
print(image_link)
space_image=image_link.a.img["src"]
space_image


C:\Users\13479\Anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


<figure class="lede">
<a href="/spaceimages/images/largesize/PIA14254_hires.jpg"><img alt="This photograph of the NASA Mars Science Laboratory rover, Curiosity, was taken during mobility testing on June 3, 2011. The location is inside the Spacecraft Assembly Facility at NASA's Jet Propulsion Laboratory, Pasadena, Calif." class="main_image" src="/spaceimages/images/largesize/PIA14254_hires.jpg" title="This photograph of the NASA Mars Science Laboratory rover, Curiosity, was taken during mobility testing on June 3, 2011. The location is inside the Spacecraft Assembly Facility at NASA's Jet Propulsion Laboratory, Pasadena, Calif."/></a>
</figure>


'/spaceimages/images/largesize/PIA14254_hires.jpg'

In [7]:
# fetch featured image link
featured_image_url = "https://www.jpl.nasa.gov" + space_image
print(featured_image_url)
mars["featured_image_url"]=featured_image_url

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_hires.jpg


In [8]:
# mars facts table to be scraped
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
mars_facts_df = tables[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
# scrapped data into pandas data frame

mars_facts_df.columns = ["Description", "Value"]
mars_facts_df
mars_facts_df.set_index('Description', inplace=True)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [11]:
# converting pandas dataframe to a html page

mars_html_table = mars_facts_df.to_html()
mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [12]:
# formatting html
mars_html_table = mars_html_table.replace('\n', '')

#print(mars_html_table)
mars["facts"] = mars_html_table
mars

{'news_title': 'Mars Now',
 'news_p': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_hires.jpg',
 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>

In [13]:
# Mars hemisphere name and image to be scraped
# usgs_url = 'https://astrogeology.usgs.gov'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)
hemispheres_html = browser.html
hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')

headers=[]
titles = hemispheres_soup.find_all('h3')
time.sleep(5)
for title in titles:
    headers.append(title.text)
images=[]
count=0
for thumb in headers:
    browser.find_by_css('img.thumb')[count].click()
    images.append(browser.find_by_text('Sample')['href'])
    browser.back()
    count=count+1
hemisphere_image_urls = []
counter = 0
for item in images:
    hemisphere_image_urls.append({"title":headers[counter],"img_url":images[counter]})
    counter=counter+1
browser.back()
time.sleep(5)
mars["hemispheres"] = hemisphere_image_urls

mars


{'news_title': 'Mars Now',
 'news_p': "NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_hires.jpg',
 'facts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>